# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers
from tensorflow.keras.optimizers import RMSprop

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
#This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(Embedding(max_features, word_embedding_dim))  
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [7]:
rmsprop = RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

model_rnn.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 16ms/step - loss: 0.6621 - accuracy: 0.6193 - val_loss: 0.6114 - val_accuracy: 0.6853
Epoch 2/10
782/782 [==============================] - 12s 15ms/step - loss: 0.5664 - accuracy: 0.7183 - val_loss: 0.5504 - val_accuracy: 0.7189
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.5057 - accuracy: 0.7572 - val_loss: 0.5164 - val_accuracy: 0.7396
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4655 - accuracy: 0.7828 - val_loss: 0.4946 - val_accuracy: 0.7543
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4360 - accuracy: 0.8004 - val_loss: 0.4775 - val_accuracy: 0.7658
Epoch 6/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4138 - accuracy: 0.8130 - val_loss: 0.4668 - val_accuracy: 0.7708
Epoch 7/10
782/782 [==============================] - 11s 13ms/step - loss: 0.3939 - accuracy: 0.8240 - val_loss: 0.4597 - val_accuracy:

In [8]:
score, acc = model_rnn.evaluate(x_test, y_test,
                                batch_size=batch_size)
print('Test loss:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 1ms/step - loss: 0.4458 - accuracy: 0.7905
Test loss: 0.4458288550376892
Test accuracy: 0.7904800176620483


## Exercise

In this exercise, we will prepare the data to use sequences of length 80 rather than length 30 and try different values of `max_features` and `word_embedding_dim` to improve the performance. 

In [9]:
maxlen = 80  # maximum length of a sequence - truncate after this
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
word_embedding_dim = 50
rnn_hidden_dim = 5

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                        kernel_initializer=initializers.RandomNormal(stddev=0.001),
                        recurrent_initializer=initializers.Identity(gain=1.0),
                        activation='relu',
                        input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

rmsprop = RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
                  optimizer=rmsprop,
                  metrics=['accuracy'])

model_rnn.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 22s 27ms/step - loss: 0.6022 - accuracy: 0.6717 - val_loss: 0.5215 - val_accuracy: 0.7364
Epoch 2/10
782/782 [==============================] - 14s 18ms/step - loss: 0.4565 - accuracy: 0.7865 - val_loss: 0.4707 - val_accuracy: 0.7820
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3883 - accuracy: 0.8278 - val_loss: 0.4487 - val_accuracy: 0.7858
Epoch 4/10
782/782 [==============================] - 23s 30ms/step - loss: 0.3470 - accuracy: 0.8517 - val_loss: 0.4289 - val_accuracy: 0.8062
Epoch 5/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3208 - accuracy: 0.8656 - val_loss: 0.3946 - val_accuracy: 0.8220
Epoch 6/10
782/782 [==============================] - 20s 25ms/step - loss: 0.3007 - accuracy: 0.8756 - val_loss: 0.4166 - val_accuracy: 0.8148
Epoch 7/10
782/782 [==============================] - 18s 24ms/step - loss: 0.2852 - accuracy: 0.8834 - val_loss: 0.3753 - val_accuracy:

In [10]:
# Lowering the most common words (max_features) & word embedding dim
max_features = 5000  
word_embedding_dim = 20
maxlen = 80  
rnn_hidden_dim = 5

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                        kernel_initializer=initializers.RandomNormal(stddev=0.001),
                        recurrent_initializer=initializers.Identity(gain=1.0),
                        activation='relu',
                        input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

rmsprop = RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
                  optimizer=rmsprop,
                  metrics=['accuracy'])

model_rnn.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 13ms/step - loss: 0.6712 - accuracy: 0.5608 - val_loss: 0.6249 - val_accuracy: 0.6844
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5959 - accuracy: 0.7316 - val_loss: 0.5921 - val_accuracy: 0.7294
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5616 - accuracy: 0.7855 - val_loss: 0.5991 - val_accuracy: 0.7226
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5392 - accuracy: 0.8068 - val_loss: 0.5508 - val_accuracy: 0.8026
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.5218 - accuracy: 0.8192 - val_loss: 0.5376 - val_accuracy: 0.8076
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4290 - accuracy: 0.8148 - val_loss: 0.4423 - val_accuracy: 0.7988
Epoch 7/10
782/782 [==============================] - 11s 15ms/step - loss: 0.3906 - accuracy: 0.8271 - val_loss: 0.4197 - val_accuracy:

In [11]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3358 - accuracy: 0.8567 - val_loss: 0.3982 - val_accuracy: 0.8222
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3283 - accuracy: 0.8589 - val_loss: 0.3894 - val_accuracy: 0.8233
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3224 - accuracy: 0.8633 - val_loss: 0.3992 - val_accuracy: 0.8232
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3166 - accuracy: 0.8663 - val_loss: 0.3825 - val_accuracy: 0.8307
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3109 - accuracy: 0.8676 - val_loss: 0.3809 - val_accuracy: 0.8310
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3066 - accuracy: 0.8712 - val_loss: 0.3709 - val_accuracy: 0.8365
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3023 - accuracy: 0.8716 - val_loss: 0.3690 - val_accuracy:

Tuning the hyperparameters produces a lower loss and higher accuracy on both the training and validation sets.


---
### Machine Learning Foundation (C) 2020 IBM Corporation